# Data cleaning and processing

The following tasks will be performed in this notebook:
- downloading files with all the data needed to perform the analysis
- importing only relevant data and creating a single dataframe
- data cleaning
- obtaining geographical coordinates of weather stations

## Downloading the data and initial cleaning

Historical meteorological data from Polish weather stations will be needed for the analysis. The data can be found on the official IMGW website in the form of .csv files. To download all the necessary files, we will use the Beautiful Soup and requests library.

In [3]:
from bs4 import BeautifulSoup
import requests
import time

In [8]:
# URL path to the online directory where all historical weather data are stored
url_core = 'https://danepubliczne.imgw.pl/data/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/dobowe/klimat/'

# Variable storing a list with particular directory' names
years = list(range(2001,2022)) + ['1996_2000','1991_1995','1986_1990','1981_1985','1976_1980','1971_1975','1966_1970','1961_1965','1956_1960','1951_1955']

ext = 'zip'     # extension of files with data
path = 'data/'      # name of target directory for downloaded data files

# Simple function for file downloading
def download_file(url):
    r = requests.get(url)
    with open((path + filename), 'wb') as f:
        f.write(r.content)
    
    print(filename + " " + str(r.status_code))
    time.sleep(0.5)     # time interval

# Loop over a list of years to download all the data (it may take a while)
for url in years:
    response = requests.get(url_core + str(url))
    
    soup = BeautifulSoup(response.content, 'html.parser')
    time.sleep(0.5)
    
    for name in soup.select(f"a[href*={ext}]"):
        filename = name.get_text().strip()
        download_url = url_core + str(url) + "/" + filename
        
        download_file(download_url)

2001_01_k.zip 200
2001_02_k.zip 200
2001_03_k.zip 200
2001_04_k.zip 200
2001_05_k.zip 200
2001_06_k.zip 200
2001_07_k.zip 200
2001_08_k.zip 200
2001_09_k.zip 200
2001_10_k.zip 200
2001_11_k.zip 200
2001_12_k.zip 200
2002_01_k.zip 200
2002_02_k.zip 200
2002_03_k.zip 200
2002_04_k.zip 200
2002_05_k.zip 200
2002_06_k.zip 200
2002_07_k.zip 200
2002_08_k.zip 200
2002_09_k.zip 200
2002_10_k.zip 200
2002_11_k.zip 200
2002_12_k.zip 200
2003_01_k.zip 200
2003_02_k.zip 200
2003_03_k.zip 200
2003_04_k.zip 200
2003_05_k.zip 200
2003_06_k.zip 200
2003_07_k.zip 200
2003_08_k.zip 200
2003_09_k.zip 200
2003_10_k.zip 200
2003_11_k.zip 200
2003_12_k.zip 200
2004_01_k.zip 200
2004_02_k.zip 200
2004_03_k.zip 200
2004_04_k.zip 200
2004_05_k.zip 200
2004_06_k.zip 200
2004_07_k.zip 200
2004_08_k.zip 200
2004_09_k.zip 200
2004_10_k.zip 200
2004_11_k.zip 200
2004_12_k.zip 200
2005_01_k.zip 200
2005_02_k.zip 200
2005_03_k.zip 200
2005_04_k.zip 200
2005_05_k.zip 200
2005_06_k.zip 200
2005_07_k.zip 200
2005_08_k.

Unzipping all files and removing remaining .zip files

In [2]:
import zipfile, os, glob

In [29]:
dir_name = 'data'
extension = ".zip"

file_list = os.listdir(dir_name)

os.chdir(dir_name) 

for item in file_list: 
    if item.endswith(extension): 
        file_name = os.path.abspath(item) 
        zip_ref = zipfile.ZipFile(file_name) 
        zip_ref.extractall(dir_name) 
        zip_ref.close() 
        os.remove(file_name) 



Each .zip file has two .csv files inside. We are only interested in files with 'k_d_' prefix so other files with 'k_d_t_' prefix may be removed.

In [33]:
fileList = glob.glob('data/k_d_t_*.csv')

for filePath in fileList:
    os.remove(filePath)

### Creating one dataframe with data from all .csv files

There is no need to combine data from all .csv files into one big file, because we are only interested in data from one day (December 25th). Therefore, we will concatenate only the rows that meet this condition into one large dataframe.

In [2]:
import pandas as pd

Quick preview of the random single .csv file. We need to upload all of the data into one dataframe with correct columns names which can be found in the dataset documentation. 

In [6]:
pd.read_csv('data/k_d_01_2005.csv', encoding='latin2').sample(5)

,249180010,PSZCZYNA,2005,01,01.1,5.4,Unnamed: 6,-1.3,Unnamed: 8,2.4,Unnamed: 10,-.1,Unnamed: 12,1.5,Unnamed: 14,W,0,9
1413,250150150,SZKLARSKA PORĘBA,2005,1,20,1.9,NaN,-2.0,NaN,-0.2,NaN,0.0,8.0,33.5,NaN,S,13,NaN
335,249190480,LALIKI,2005,1,27,-6.0,NaN,-10.2,NaN,-7.9,NaN,0.0,8.0,2.3,NaN,S,100,NaN
3295,251220070,SOBIESZYN,2005,1,11,9.3,NaN,4.4,NaN,6.4,NaN,1.4,NaN,0.0,9.0,NaN,0,9.0
383,249190560,JABŁONKA,2005,1,13,3.4,NaN,-0.3,NaN,1.7,NaN,0.0,8.0,2.7,NaN,S,0,9.0
1751,250170110,DOBROGOSZCZ,2005,1,17,3.7,NaN,-3.9,NaN,0.5,NaN,-4.8,NaN,0.0,9.0,NaN,0,9.0


Iterating over all .csv files and conacetating into one dataframe with records only for December 25th.

In [17]:
# Correct columns names found in the dataset documentation
colnames = ['station_id','station_name','year','month','day','t_max','t_max_status','t_min','t_min_status','t_mean','t_mean_status','t_ground','t_ground_status','percip','percip_status','percip_type','snow','snow_status']
df = pd.DataFrame()

for i in glob.glob('data/k_d*.csv'):
    temp = pd.read_csv(i, encoding='latin2', names=colnames)
    temp = temp.loc[(temp['month']==12) & (temp['day']==25)]
    df = pd.concat([df, temp])
    temp = pd.DataFrame()

Quick preview of the dataframe 

In [19]:
df.sample(10)

,station_id,station_name,year,month,day,t_max,t_max_status,t_min,t_min_status,t_mean,t_mean_status,t_ground,t_ground_status,percip,percip_status,percip_type,snow,snow_status
3558,252170210,KÓRNIK,2011,12,25,5.0,NaN,3.0,NaN,4.1,NaN,0.4,NaN,0.4,NaN,W,0,9.0
1884,253230020,RÓŻANYSTOK,2019,12,25,5.2,NaN,1.6,NaN,3.7,NaN,2.0,NaN,0.1,NaN,W,0,9.0
18243,250200120,SKRONIÓW,1963,12,25,1.1,NaN,-18.1,NaN,-5.0,NaN,0.0,8.0,0.0,NaN,W,6,NaN
19521,350220580,RZESZÓW-JASIONKA,1953,12,25,-4.1,NaN,-15.9,NaN,-6.6,NaN,-20.0,NaN,0.0,9.0,NaN,5,NaN
27914,251170290,NAMYSŁÓW,1982,12,25,0.0,NaN,-4.1,NaN,-2.2,NaN,-4.1,NaN,1.5,NaN,S,0,9.0
5255,249190370,RABKA,1991,12,25,0.3,NaN,-6.0,NaN,-4.0,NaN,-9.0,NaN,0.9,NaN,S,20,NaN
62957,353160215,SZCZECINEK,1962,12,25,-4.9,NaN,-11.5,NaN,-6.1,NaN,0.0,8.0,0.1,NaN,S,5,NaN
31042,251210120,PUŁAWY,1984,12,25,-2.2,NaN,-7.7,NaN,-5.6,NaN,-8.3,NaN,0.0,NaN,S,0,9.0
7679,249200320,ŁOPUSZNA,1988,12,25,2.8,NaN,-1.6,NaN,-0.3,NaN,-1.1,NaN,1.0,NaN,S,23,NaN
27967,251170290,NAMYSŁÓW,1981,12,25,-0.6,NaN,-5.5,NaN,-1.8,NaN,-11.3,NaN,7.2,NaN,S,10,NaN


Saving dataframe to a .csv file.

In [20]:
df.to_csv('data/snow_data.csv')

## Further data cleaning and preparing

The main goal is to answer the question of what is the statistical chance of snow on Christmas Day (on December 25th). Therefore, a new dataframe needs to be created where all years of snow occurrence are counted and divided by the total number of years of measurements (specific to each station). <br>

Then we need to determine the geographical coordinates of each station, which will allow us to create a map with areas with a certain chance of snow.

#### Stations names

During the initial browsing of the data, it was noticed that some of the weather stations names contained incorrect characters, meaning that it did not import correctly. Let's fix that problem.

In [63]:
# Displaying only unique stations names in the dataframe
df['station_name'].unique()

array(['PSZCZYNA', 'CIESZYN', 'BRENNA', 'WISŁA', 'ISTEBNA-KUBALONKA',
       'INWAŁD', 'MIĘDZYBRODZIE BIALSKIE', 'MAKÓW PODHALAŃSKI',
       'NOWY DWÓR', 'ZAWOJA', 'RABKA', 'LALIKI', 'OBIDOWA', 'JABŁONKA',
       'HALA ORNAK', 'ŁAZY', 'DOBCZYCE', 'JASTRZĘBIA', 'LIMANOWA',
       'PTASZKOWA', 'ŁĽCKO', 'ŁOPUSZNA', 'MIZERNA', 'KRO\x8cCIENKO',
       'PIWNICZNA', 'NIEDZICA', 'KRYNICA', 'BUKOWINA TATRZAŃSKA',
       'MUSZYNA', 'PORONIN', 'HALA GĽSIENICOWA', 'DOLINA PIĘCIU STAWÓW',
       'BIECZ-GRUDNA', 'DUKLA', 'WYSOWA', 'BARWINEK', 'DYNÓW',
       'SANOK-TREPCZA', 'SOLINA-JAWOR', 'BALIGRÓD-MCHAWA', 'KOMAŃCZA',
       'TERKA', 'STUPOSIANY', 'BOGATYNIA', '\x8cWIERADÓW-ZDRÓJ',
       'SZKLARSKA PORĘBA', 'JAKUSZYCE', 'KARPACZ', 'PAPROTKI', 'PSZENNO',
       'SZCZAWNO-ZDRÓJ', 'TARNÓW', 'SŁOSZÓW', 'LĽDEK-ZDRÓJ',
       'DŁUGOPOLE-ZDRÓJ', 'BOLESŁAWÓW', 'DOBROGOSZCZ', 'GRODKÓW',
       'KORFANTÓW', 'OTMUCHÓW', 'GŁUCHOŁAZY', 'GŁUBCZYCE', 'STARE OLESNO',
       'KOCHCICE', '\x8cWIERKLANIEC', 'SUKOW

In [64]:
# Replacing unwanted strings with the correct letter in a Polish alphabet

df.station_name = df.station_name.str.replace('\x8c','Ś')
df.station_name = df.station_name.str.replace('Ľ','Ą')
df.station_name = df.station_name.str.replace('\x8f','Ź')

In [65]:
df['station_name'].unique()

array(['PSZCZYNA', 'CIESZYN', 'BRENNA', 'WISŁA', 'ISTEBNA-KUBALONKA',
       'INWAŁD', 'MIĘDZYBRODZIE BIALSKIE', 'MAKÓW PODHALAŃSKI',
       'NOWY DWÓR', 'ZAWOJA', 'RABKA', 'LALIKI', 'OBIDOWA', 'JABŁONKA',
       'HALA ORNAK', 'ŁAZY', 'DOBCZYCE', 'JASTRZĘBIA', 'LIMANOWA',
       'PTASZKOWA', 'ŁĄCKO', 'ŁOPUSZNA', 'MIZERNA', 'KROŚCIENKO',
       'PIWNICZNA', 'NIEDZICA', 'KRYNICA', 'BUKOWINA TATRZAŃSKA',
       'MUSZYNA', 'PORONIN', 'HALA GĄSIENICOWA', 'DOLINA PIĘCIU STAWÓW',
       'BIECZ-GRUDNA', 'DUKLA', 'WYSOWA', 'BARWINEK', 'DYNÓW',
       'SANOK-TREPCZA', 'SOLINA-JAWOR', 'BALIGRÓD-MCHAWA', 'KOMAŃCZA',
       'TERKA', 'STUPOSIANY', 'BOGATYNIA', 'ŚWIERADÓW-ZDRÓJ',
       'SZKLARSKA PORĘBA', 'JAKUSZYCE', 'KARPACZ', 'PAPROTKI', 'PSZENNO',
       'SZCZAWNO-ZDRÓJ', 'TARNÓW', 'SŁOSZÓW', 'LĄDEK-ZDRÓJ',
       'DŁUGOPOLE-ZDRÓJ', 'BOLESŁAWÓW', 'DOBROGOSZCZ', 'GRODKÓW',
       'KORFANTÓW', 'OTMUCHÓW', 'GŁUCHOŁAZY', 'GŁUBCZYCE', 'STARE OLESNO',
       'KOCHCICE', 'ŚWIERKLANIEC', 'SUKOWICE', 'SI

#### Dropping unnecessary columns

The main dataframe has many columns, but we are interested only in a few like: station_id, station_name, year, snow and t_mean.

In [66]:
df.sample(1)

,Unnamed: 0,station_id,station_name,year,month,day,t_max,t_max_status,t_min,t_min_status,t_mean,t_mean_status,t_ground,t_ground_status,percip,percip_status,percip_type,snow,snow_status
2667,1078,251160320,RADZYŃ,2021,12,25,0.7,NaN,-8.9,NaN,-5.9,NaN,-11.5,NaN,0.0,9.0,NaN,1,NaN


In [70]:
df = df[['station_id','station_name','year','snow','t_mean']]
df

,station_id,station_name,year,snow,t_mean
0,249180010,PSZCZYNA,2001,18,-5.8
1,249180130,CIESZYN,2001,24,-1.6
2,249180160,BRENNA,2001,39,-3.7
3,249180230,WISŁA,2001,70,-7.0
4,249180260,ISTEBNA-KUBALONKA,2001,97,-8.7
...,...,...,...,...,...
10055,254190050,FROMBORK,2000,0,-1.6
10056,254190120,KMIECIN,2000,3,-1.5
10057,254200080,LIDZBARK WARMIŃSKI,2000,0,-2.3
10058,254220030,GOŁDAP,2000,3,-2.7


### Finding the coordinates of weather stations

For further analysis, we'll want to create a map with areas showing the chances of snow on December 25. For this we need the geographical coordinates of each weather station.

Unfortunately, this data is not in the main dataframe. After a long search, I managed to come across a pdf document on the IMGW website (IMGW - National Polish Meteo Institute, provider of the original dataset) with a list of coordinates for all stations.

To read the pdf document, we will use the tabula library, which can convert pdf to a dataframe.

In [1]:
import tabula

Reading a pdf file into a dataframe.

In [72]:
c = tabula.read_pdf('mapa_zawartosci_klimat.pdf', pages='all', stream=True)

Got stderr: gru 16, 2022 9:29:33 PM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider loadDiskCache
gru 16, 2022 9:29:33 PM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider <init>
gru 16, 2022 9:29:34 PM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider <init>



In [80]:
c[0].sample(5)

,Kod METEO Nazwa stacji,Dł.g.,Sz.g.,Nazwa rzeki,1951-1960 1961-1970,1971-1980,1981-1990,1991-2000,2001-2010,2011-2020,2021-2030
1,3150 BABIMOST,15 47,52 08,OBRZYCA,---------- ---------- ---------- -----KKKK- --...,NaN,NaN,NaN,NaN,NaN,NaN
31,2450 BYDGOSZCZ,18 03,53 10,BRDA,---------- ---------- ---------- ---------- -N...,NaN,NaN,NaN,NaN,NaN,NaN
8,1101 BIAŁOGARD,15 59,54 00,PARSĘTA,---------- ---------- ---------- -------KKK KN...,NaN,NaN,NaN,NaN,NaN,NaN
14,600 BIELSKO-BIAŁA,19 00 04,49 48 29,BIAŁA,KKKKKKKKKK KKKKK----- ---------- ---------- --...,NaN,NaN,NaN,NaN,NaN,NaN
40,3401 CIECHOCINEK,18 48,52 53,WISŁA,---------N KKKKKKKKKK KKKKKKKKNN KNNNNKKN-- --...,NaN,NaN,NaN,NaN,NaN,NaN


The converted data is in one dataframe. Each page of the pdf document corresponds to one table. There are 9 tables in total. It turned out that table #8 has the wrong column headings that need to be changed to match the rest.

In [81]:
c[7].sample(3)

,5101,ŚWIERADÓW-ZDRÓJ,15 20,50 54,KWISA,---------- ---------- ---------- --------KK KKKKKKKKKK KKKKKKKKKK KKKK------ ----------
16,2002,TRZEBIEŻ,14 32,53 40,ZALEW SZCZECIŃSKI,KKKKKKKKKK KKKKKKKKKK KKKKKKKKKK KKKKKKKKKK --...
12,3508,TOPOLA-BŁONIE,19 11,52 05,NER,---------- KKKKKKKKKK KKKKKKKKKK KKKKKKNKN- --...
9,1502,TOLKMICKO,19 32,54 19,ZALEW WIŚLANY,KKKKKKKKKK KKKKKKKKKK KKKKKKKKKK ---------- --...


In [82]:
# Renaming three key columns
c[7].rename(columns = {'ŚWIERADÓW-ZDRÓJ':'Kod METEO Nazwa stacji', '15 20':'Dł.g.', '50 54':'Sz.g.'}, inplace = True)


In [83]:
c[7].sample(3)

,5101,Kod METEO Nazwa stacji,Dł.g.,Sz.g.,KWISA,---------- ---------- ---------- --------KK KKKKKKKKKK KKKKKKKKKK KKKK------ ----------
14,3001,TRZCIŃSKO-ZDRÓJ,14 36,52 58,TYWA,---------- ---------- ---------- ---------K KK...
17,4901,UHNIN,23 02,51 34,TYŚMIENICA,---------- ---------- ---------- -------KKK KK...
19,6502,WADOWICE,19 31,49 53,SKAWA,---------- KKKKKKKKKK KKKKKKKKKK ---------- --...


Only the first three columns corresponding to station name, longitude and latitude are useful for our purposes. Therefore, we will combine them into one dataframe.

In [84]:
pdList = [c[0].iloc[:,0:3], c[1].iloc[:,0:3], c[2].iloc[:,0:3], c[3].iloc[:,0:3], c[4].iloc[:,0:3], c[5].iloc[:,0:3], c[6].iloc[:,0:3], c[7].iloc[:,1:4], c[8].iloc[:,0:3]]

stations = pd.concat(pdList).reset_index(drop=True)

In [87]:
# Changing columns names
stations.rename(columns = {'Kod METEO Nazwa stacji':'station_name', 'Dł.g.':'lon', 'Sz.g.':'lat'}, inplace = True)

stations.head(5)

,station_name,lon,lat
0,NaN,NaN,NaN
1,3150 BABIMOST,15 47,52 08
2,3152 BABIMOST,15 47 46,52 08 24
3,6808 BALIGRÓD-MCHAWA,22 17,49 21
4,6708 BARWINEK,21 41,49 26


The next step is to check for duplicates, remove rows with NaNs, remove unnecessary numbers in front of the station name.

In [88]:
# List all rows with NaN values

stations[stations.isna().any(axis=1)]

,station_name,lon,lat
0,NaN,NaN,NaN
41,NaN,NaN,NaN
82,NaN,NaN,NaN
123,NaN,NaN,NaN
164,NaN,NaN,NaN
205,NaN,NaN,NaN
246,NaN,NaN,NaN
326,NaN,NaN,NaN


In [89]:
# Drop all rows with NaN

stations = stations.dropna().reset_index(drop=True)

In [92]:
# Remove any number and signs before the station name

stations['station_name'] = stations['station_name'].str.replace('^\d+\s+', '')

C:\Users\Wojtek\AppData\Local\Temp\ipykernel_2996\231100400.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  stations['station_name'] = stations['station_name'].str.replace('^\d+\s+', '')


In [93]:
stations.head(5)

,station_name,lon,lat
0,BABIMOST,15 47,52 08
1,BABIMOST,15 47 46,52 08 24
2,BALIGRÓD-MCHAWA,22 17,49 21
3,BARWINEK,21 41,49 26
4,BEŁCHATÓW,19 22,51 24


In [94]:
# List all station duplicates

stations[stations.duplicated(['station_name'], keep=False)]

,station_name,lon,lat
0,BABIMOST,15 47,52 08
1,BABIMOST,15 47 46,52 08 24
29,BYDGOSZCZ,17 58,53 06
30,BYDGOSZCZ,18 03,53 10
62,GDAŃSK-RĘBIECHOWO,18 28,54 23
63,GDAŃSK-RĘBIECHOWO,18 28,54 23
78,GRABOWNICA,17 27,51 28
79,GRABOWNICA,22 05,49 40
170,MSZANA DOLNA,20 05,49 40
171,MSZANA DOLNA,20 03 16,49 39 49


In [95]:
# Remove all duplicates

stations = stations.drop_duplicates(subset=['station_name'])
stations.head(5)

,station_name,lon,lat
0,BABIMOST,15 47,52 08
2,BALIGRÓD-MCHAWA,22 17,49 21
3,BARWINEK,21 41,49 26
4,BEŁCHATÓW,19 22,51 24
5,BEZEK,23 16,51 11


In [98]:
# Reseting index

stations = stations.reset_index(drop=True)
stations

,station_name,lon,lat
0,BABIMOST,15 47,52 08
1,BALIGRÓD-MCHAWA,22 17,49 21
2,BARWINEK,21 41,49 26
3,BEŁCHATÓW,19 22,51 24
4,BEZEK,23 16,51 11
...,...,...,...
323,ZIELONA GÓRA,15 31 28,51 55 49
324,ZYBISZÓW,16 55,51 04
325,ŻARNOWA,21 49,49 53
326,ŻUBRACZE,22 16 07,49 12 23


The last step is to convert the current data with coordinates (format: degrees, minutes, seconds) to decimal degrees format.

In [101]:
import re

# Simple function which converts latitude/longitude from degrees, minutes, seconds to decimal degrees

def latlon_parser(data):
    data = re.sub('[^A-Za-z0-9]+', ' ', data).rstrip()
    if len(re.split(' ', data)) > 2:
        deg, minutes, seconds =  re.split(' ', data)
        return round((float(deg) + float(minutes)/60 + float(seconds)/(60*60)),6) 
    else:
        deg, minutes =  re.split(' ', data)
        return round((float(deg) + float(minutes)/60),6) 

In [102]:
stations['lon'] = stations['lon'].map(latlon_parser)
stations['lat'] = stations['lat'].map(latlon_parser)

In [103]:
stations.sample(5)

,station_name,lon,lat
97,KALISZ,18.081944,51.781944
273,ŚWIĘTAJNO,21.227778,53.563333
203,POŚWIĘTNE,20.383333,52.633333
266,SZKLARSKA PORĘBA,15.533333,50.833333
325,ŻARNOWA,21.816667,49.883333


## Merging the main dataframe with longitude/latitude data

The last step will be to merge the coordinates of each station to the main dataframe.

In [107]:
df = df.merge(stations, how='left', on='station_name')
df.sample(5)

,station_id,station_name,year,snow,t_mean,lon,lat
2791,252180080,KOŁUDA WIELKA,1952,5,1.1,18.150000,52.733333
6683,249190370,RABKA,1981,25,-1.1,19.966667,49.616667
1977,250200340,JĘDRZEJÓW-SUDÓŁ,2013,0,4.8,20.275556,50.653056
2859,354160105,KOSZALIN,1952,0,2.7,16.155556,54.204444
869,250220030,WYSOKIE,2006,0,0.3,22.666667,50.916667


Let's check is there any NaN values in the new dataframe.

In [108]:
df[df.isna().any(axis=1)]

,station_id,station_name,year,snow,t_mean,lon,lat
44,250150090,ŚWIERADÓW-ZDRÓJ,2001,65,-1.8,NaN,NaN
201,250150090,ŚWIERADÓW-ZDRÓJ,2002,10,-2.9,NaN,NaN
357,250150090,ŚWIERADÓW-ZDRÓJ,2003,15,-2.7,NaN,NaN
511,250150090,ŚWIERADÓW-ZDRÓJ,2004,0,4.9,NaN,NaN
669,250150090,ŚWIERADÓW-ZDRÓJ,2005,9,-1.2,NaN,NaN
830,250150090,ŚWIERADÓW-ZDRÓJ,2006,0,0.2,NaN,NaN
991,250150090,ŚWIERADÓW-ZDRÓJ,2007,0,-4.3,NaN,NaN
1152,250150090,ŚWIERADÓW-ZDRÓJ,2008,0,-2.3,NaN,NaN
1313,250150090,ŚWIERADÓW-ZDRÓJ,2009,0,6.8,NaN,NaN
1472,250150090,ŚWIERADÓW-ZDRÓJ,2010,53,-4.2,NaN,NaN


In [110]:
# Displaying unique stations with NaN values

df[df.isna().any(axis=1)]['station_name'].unique()

array(['ŚWIERADÓW-ZDRÓJ', 'KATOWICE', 'WROCŁAW', 'ŁÓDŹ', 'POZNAŃ',
       'WARSZAWA', 'KOŁOBRZEG'], dtype=object)

The geographical coordinates of 7 stations are missing. Fortunately, these are large and known cities in Poland whose coordinates can be easily found.

In [111]:
# Creating a simple dataframe with geographical coordinates for each missing stations

cities = pd.DataFrame({
    'station_name': ['WARSZAWA', 'KATOWICE', 'WROCŁAW', 'ŁÓDŹ', 'POZNAŃ', 'KOŁOBRZEG','ŚWIERADÓW-ZDRÓJ'],
    'lon': [21.003886, 19.0241157, 17.031836, 19.455495, 16.9331527, 15.575616, 15.334718],
    'lat': [52.2316535, 50.256350, 51.1106167, 51.767715, 52.4079809, 54.176423, 50.909222]
    })
cities

,station_name,lon,lat
0,WARSZAWA,21.003886,52.231654
1,KATOWICE,19.024116,50.256350
2,WROCŁAW,17.031836,51.110617
3,ŁÓDŹ,19.455495,51.767715
4,POZNAŃ,16.933153,52.407981
5,KOŁOBRZEG,15.575616,54.176423
6,ŚWIERADÓW-ZDRÓJ,15.334718,50.909222


We add the missing stations to the full list and merge them again to the main dataframe.

In [112]:
stations = pd.concat([stations, cities], ignore_index=True)
stations

,station_name,lon,lat
0,BABIMOST,15.783333,52.133333
1,BALIGRÓD-MCHAWA,22.283333,49.350000
2,BARWINEK,21.683333,49.433333
3,BEŁCHATÓW,19.366667,51.400000
4,BEZEK,23.266667,51.183333
...,...,...,...
330,WROCŁAW,17.031836,51.110617
331,ŁÓDŹ,19.455495,51.767715
332,POZNAŃ,16.933153,52.407981
333,KOŁOBRZEG,15.575616,54.176423


In [117]:
df = df.merge(stations, how='left', on='station_name')

Let's check if there are any missing data.

In [118]:
df[df.isna().any(axis=1)]

,station_id,station_name,year,snow,t_mean,lon,lat


In [119]:
# Saving dataframe with all cleaned data to a csv file

df.to_csv('data/snow_data_all_filtered.csv')